In [1]:
import requests
import re
import time
import json
import random
import pandas as pd
import datetime

Функция для получения данных

In [2]:
proxy = {
    'http': 'http://46.47.197.210:3128',
    'http': 'http://62.33.207.201:80',
    'http': 'http://91.247.250.215:4145',
    'http': 'http://84.53.239.95:4145',
    'http': 'http://178.35.177.242:3629',
    'http': 'http://109.94.182.128:4145',
    'http': 'http://178.176.193.56:1080'
}

def get_api_data(url, params = '', headers ='', proxy = ''): 
    r = requests.get(url, params, headers=headers, proxies=proxy)
    return r.json()

Функция для случайного выбора юзер-агента

In [3]:
def choose_user_agent(user_agents):
    user_agent = user_agents[random.randrange(len(user_agents))]
    headers = {'user-agent': user_agent}
    return headers

Чтение данных юзерагетов

In [4]:
with open(r'C:\Users\Lenovo\Desktop\diplom\desktop_user_agent.txt', 'r', encoding='utf-8') as fp:
    user_agents = [line.strip() for line in fp.read().split('\n')]


Функция для получения данных о вакансиях за день

In [5]:
def get_vac_ids(date):
    url = r'https://api.hh.ru/vacancies'
    all_vac = []
    par = {'text': 'аналитик', 'area': 113, 'search_field': 'name', 'date_from': str(date), 'date_to': str(date)}
    data = get_api_data(url, params=par, headers=choose_user_agent(user_agents))
    pages = data['pages']
    for page in range(pages):
        par = {'text': 'аналитик', 'area': 113, 'search_field': 'name', 'date_from': str(date), 'date_to': str(date), 'page': page}
        data = get_api_data(url, params=par, headers=choose_user_agent(user_agents))
        all_vac = all_vac + data['items']
    vac_ids = []
    for one_vac in all_vac:
        vac_ids.append(one_vac['id'])
    vac_ids.sort()
    return vac_ids

Проверка нет ли капчи

In [6]:
url = f'https://api.hh.ru/vacancies/42018113'
vac_descr = get_api_data(url, headers=choose_user_agent(user_agents))
area = vac_descr['area']['name']
description = re.sub(r'\<[^>]*\>', '', vac_descr['description'])
employment = vac_descr['employment']['name']
experience = vac_descr['experience']['name']
id = vac_descr['id']
key_skills = vac_descr['key_skills']
if len(vac_descr['key_skills']) > 0:
    # if len(vac_descr['key_skills'][0]) > 0:
    skills = []
    for skill in vac_descr['key_skills']:
        skills.append(skill['name'])
    key_skills = skills
name = vac_descr['name']
publication_date = vac_descr['published_at'][0:10]
salary = vac_descr['salary']
schedule = vac_descr['schedule']
specializations = vac_descr['specializations']
if len(vac_descr['professional_roles']) > 0:
    prof_roles = []
    for role in vac_descr['professional_roles']:
        prof_roles.append(role['name'])
    proffesional_roles = prof_roles
employer = vac_descr['employer']['name']
print(salary)

None


Получение нужных данных о вакансиях

In [7]:
def get_vac_data(vac_ids):
    vac_info = []
    for one_vac in vac_ids:
        url = f'https://api.hh.ru/vacancies/{one_vac}'
        vac_descr = get_api_data(url, headers=choose_user_agent(user_agents))
        #print(vac_descr)
        try:
            area = vac_descr['area']['name']
            description = re.sub(r'\<[^>]*\>', '', vac_descr['description'])
            employment = vac_descr['employment']['name']
            experience = vac_descr['experience']['name']
            id = vac_descr['id']
            key_skills = vac_descr['key_skills']
            if len(vac_descr['key_skills']) > 0:
                skills = []
                for skill in vac_descr['key_skills']:
                    skills.append(skill['name'])
                key_skills = skills
            name = vac_descr['name']
            publication_date = vac_descr['published_at'][0:10]
            salary = vac_descr['salary']
            schedule = vac_descr['schedule']['name']
            specializations = vac_descr['specializations']
            if len(vac_descr['professional_roles']) > 0:
                prof_roles = []
                for role in vac_descr['professional_roles']:
                    prof_roles.append(role['name'])
                proffesional_roles = prof_roles
            employer = vac_descr['employer']['name']
        except:
            print(one_vac, 'ERRROR')
            continue
        vac_info.append({
            'area': area,
            'description': description,
            'employment': employment,
            'experience': experience,
            'id': id,
            'key_skills': key_skills,
            'name': name,
            'publication_date': publication_date,
            'salary': salary,
            'schedule': schedule,
            'specializations': specializations,
            'proffesional_roles': proffesional_roles,
            'employer': employer,
            'source': 'hh.ru'
        })
        time.sleep(0.5)
    return vac_info


Функция для дописывания полученные данных за день в файл

In [8]:
def add_data_to_file(vac_info, path):
    with open(path + '.json', 'r', encoding='utf-8') as fp:
        all_vacs = json.load(fp)
    all_vacs = all_vacs + vac_info
    with open(path + '.json', 'w', encoding='utf-8') as fp:
        json.dump(all_vacs, fp, ensure_ascii=False)

Функция, чтобы сохранить из json в excel

In [38]:
def from_json_to_excel(path):
    with open(path + '.json', 'r', encoding='utf-8') as fp:
        to_excel = json.load(fp)
    df = pd.DataFrame(to_excel)
    df.to_excel(path + '.xlsx')

Функция для добавления навыков в словарь и сохранения в файл (тут собираются уникальные значаения, этот в итоге не нужно)

In [10]:
def add_skills_to_dict(vac_info, save, path):
    with open(path + '.json', 'r', encoding='utf-8') as fp:
        skills_dict = json.load(fp)
    for i, vac in enumerate(vac_info):
        if len(vac['key_skills']) > 0:
            for skill in vac['key_skills']:
                if skill.lower() not in skills_dict:
                    skills_dict.append(skill.lower())
    skills_dict.sort()
    if save == True:
        with open(path + '.json', 'w', encoding='utf-8') as fp:
            json.dump(skills_dict, fp, ensure_ascii=False)
    return skills_dict

In [11]:
ids = get_vac_ids(datetime.date.today() - datetime.timedelta(1))

In [12]:
vac_info = get_vac_data(ids)

In [13]:
print(vac_info[0]['publication_date'])
print(vac_info[0])
pprint(len(vac_info))

2023-03-25
{'area': 'Пенза', 'description': 'Наша организация является аккредитованной IT компанией, сопровождающей критическую инфраструктуру.Наша организация – объединение ведущих компаний для создания масштабной структуры по разработке информационных систем и созданию ИТ-продуктов. Мы предоставляем заказчикам масштабные ИТ-решения. Разрабатываем гибкие системы автоматизации управления предприятиями ключевых отраслей экономики, поставляем лицензионное оборудование и производим собственные устройства. Наша компания является ведущим интегратором в области, и мы можем предоставить своим клиентам проекты, отвечающие последним технологическим достижениям. Требования:  Высшее техническое образование; Опыт работы от 3 лет; Умение анализировать техническую документацию; Умение составлять техническую документацию; Умение преобразовывать требования заказчика в задачи для разработчиков.  Приветствуется:  Опыт проектирования баз данных; Знание SQL; Знание SOAP, xsd; Знание одной из нотаций модел

In [14]:
path = r'C:\Users\Lenovo\Desktop\diplom\hh_data'
add_data_to_file(vac_info, path)

In [39]:
# path = r'C:\Users\Lenovo\Desktop\diplom\hh_data'
# from_json_to_excel(path)